In [63]:
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import pandas as pd
import numpy as np
import sklearn.preprocessing
import sklearn.metrics

In [8]:
# read in imputed data
sensor_census_imp = pd.read_csv('../data/sensor_census_imputed_rf.csv')

In [18]:
np.random.seed(1)

# get sites for val/test data
val_test_sites = np.random.choice(np.unique(sensor_census_imp['site'].values), round(len(np.unique(sensor_census_imp['site'].values))/5), replace = False)

# get sites for test data
test_sites = np.random.choice(np.unique(val_test_sites), round(len(np.unique(val_test_sites))/2), replace = False)

# train sites/rows and x/y split
sensor_census_imp_train = sensor_census_imp[~sensor_census_imp['site'].isin(val_test_sites)]
sensor_census_imp_train_x = sensor_census_imp_train.iloc[:, 2:]
sensor_census_imp_train_y = sensor_census_imp_train.iloc[:, 1]

# val sites/rows and x/y split
sensor_census_imp_val = sensor_census_imp[(sensor_census_imp['site'].isin(val_test_sites)) & (~sensor_census_imp['site'].isin(test_sites))]
sensor_census_imp_val_x = sensor_census_imp_val.iloc[:, 2:]
sensor_census_imp_val_y = sensor_census_imp_val.iloc[:, 1]

# test sites/rows and x/y split
sensor_census_imp_test = sensor_census_imp[sensor_census_imp['site'].isin(test_sites)]
sensor_census_imp_test_x = sensor_census_imp_test.iloc[:, 2:]
sensor_census_imp_test_y = sensor_census_imp_test.iloc[:, 1]

# standardize train, val, and test data
standardizer = sklearn.preprocessing.StandardScaler(with_mean = True, with_std = True)
sensor_census_imp_train_x_stand = standardizer.fit_transform(sensor_census_imp_train_x)
sensor_census_imp_val_x_stand = standardizer.transform(sensor_census_imp_val_x)
sensor_census_imp_test_x_stand = standardizer.transform(sensor_census_imp_test_x)

In [17]:
# create torch tensor tuples for train, val, test
train = TensorDataset(torch.from_numpy(sensor_census_imp_train_x_stand), torch.from_numpy(sensor_census_imp_train_y.values))
val = TensorDataset(torch.from_numpy(sensor_census_imp_val_x_stand), torch.from_numpy(sensor_census_imp_val_y.values))
test = TensorDataset(torch.from_numpy(sensor_census_imp_test_x_stand), torch.from_numpy(sensor_census_imp_test_y.values))

# create batches
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size = batch_size, shuffle = True)
validation_loader = torch.utils.data.DataLoader(dataset = val, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(dataset = test, batch_size = batch_size, shuffle = False)

In [163]:
# model architecture
ff_nn = torch.nn.Sequential()
ff_nn.add_module(name = '1st linear', module = torch.nn.Linear(in_features = sensor_census_imp_train_x_stand.shape[1], out_features = 100, bias = True)) # linear layer
ff_nn.add_module(name = 'norm', module = torch.nn.BatchNorm1d(num_features = 100)) # normalize before tanh
ff_nn.add_module(name = 'tanh', module = torch.nn.Tanh()) # tanh function for hidden layer
ff_nn.add_module(name = '2nd linear', module = torch.nn.Linear(in_features = 100, out_features = 1, bias = True)) # 2nd linear layer

# mse loss
loss_function = torch.nn.MSELoss()

# adam optimizer
adam = torch.optim.Adam(ff_nn.parameters(), lr=0.001, weight_decay=0)

In [164]:
# function for computing accuracy
def compute_accuracy(data_loader, model):
    model_pred = []
    targets = []
    for batch in data_loader:
        model_output = model(Variable(batch[0]).float()).data.numpy() # model preds
        model_pred += model_output.tolist() # add to list of preds
        targets += batch[1].numpy().tolist() # true digit values
        
    return sklearn.metrics.r2_score(targets, model_pred)

In [165]:
num_epochs = 30

for epoch in range(num_epochs):   
    for batch in train_loader:
        model_output = ff_nn(Variable(batch[0]).float()) # model predictions
        targets = Variable(batch[1].float()) # true digit values

        adam.zero_grad() # zero gradient
        loss_batch = loss_function(model_output, targets) # compute loss
        loss_batch.backward() # take the gradient wrt parameters
        adam.step() # update parameters
                    
print('Test accuracy for lambda=' + str(0) + ', learning rate=' + str(0.01) + ', batch size='\
        + str(100) + ', hidden units=' + str(100) + ': ' + str(compute_accuracy(train_loader, ff_nn))) # compute validation acc

Test accuracy for lambda=0, learning rate=0.01, batch size=100, hidden units=100: 0.839866332868
